## DTM building in Py

Class,

We've seen R's text-an in BOW functionality extensively. Time to quickly replicate the essentials of that work in py as well. 

Plan is to load a corpus, clean it up, apply stopwords and stem if required and finally build a DTM out of it. See below.

In [1]:
## setup chunk
import time   # to time 'em opns
t0 = time.time()    # start timer
import numpy as np
import pandas as pd
import nltk
from bs4 import BeautifulSoup
import re
import os
import codecs
from sklearn import feature_extraction
# import mpld3  # conda install -c conda-forge mpld3 
t1 = time.time()

time.taken = round(t1-t0, 3)
print(time.taken)
print("\n")    # print newline

72.529




### Data Loading and Basic Cleaning

The document 'synopses_wiki.txt' contains movie synopses or summaries data scraped from Wiki.

Load this document into py by customzing the path in the code below. We'll use only the top 100 summaries for demo purposes here in class. Feel free to try the same on the full dataset or on other, larger datasets. Do time your ops.

Special mention to the 'encoding="utf-8"' bit in open(). Without it, error will come as some characters in the text are non-ASCII.

Note also the use of *html.parser*from bs4 to get rid of html junk.

In [3]:
# note insertion of utf-8 enconding here below. Else charmap error comes.
synopses_wiki = open(r"C:\Users\31172\TABA\Session 1\synopses_list_wiki.txt", encoding="utf-8").read().split('\n BREAKS HERE')
synopses_wiki = synopses_wiki[:100]   # only 101 summaries taken for analysis

# use bs4 to cleanup html junk & append into clean_text
t0 = time.time()
synopses_clean_wiki = []
for text in synopses_wiki:
    text = BeautifulSoup(text, 'html.parser').getText()
    #strips html formatting and converts to unicode
    synopses_clean_wiki.append(text)
t1 = time.time()
print(round(t1-t0, 3))    # 0.036 secs

synopses_wiki = synopses_clean_wiki

0.024


### Tokenization and Stemming

Below we import NLTK's sentence and word tokenizer, and stemmer. Note the use of list comprehension to bundle both into one  line of efficient code.

Note also the use of regex from *re* to detect and drop any non alphabetic characters from the corpus.

Find below two straightforward user defined funcs to tokenize (and stem).

We will apply these funcs on each doc in the corpus subsequently.

In [4]:
# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english')

# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

## here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation) using regex
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [5]:
# Use above funcs to iterate over the list of synopses to create two vocabularies: one stemmed and one only tokenized. 
totalvocab_stemmed = []
totalvocab_tokenized = []
t0 = time.time()
for i in synopses_wiki:
    allwords_stemmed = tokenize_and_stem(i)
    totalvocab_stemmed.extend(allwords_stemmed)
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)
t1 = time.time()
print(round(t1-t0, 3))    # 10.716 secs

7.667


### Building the DTM for TFIDF weighing

While lists are nice nd all, nothing quite like the tabular structure of a dataFrame to put the mind at ease and conjure up analysis visualizations in our minds. Hence, step 1 below is building and populating a panda DF.

Next, we import *TfidfVectorizer* to build the DTM under TFIDF scheme.

Finally, we inspect the DTM object, its dimensions, the main column names etc.

In [6]:
## create a pandas DataFrame with the stemmed vocabulary as the index and the tokenized words as the column
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)

## Tf-idf and document similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
# defining parms for the tfidf-tokenizer here
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, # max proportion of docs word is present in
				   max_features=200000,
                                   min_df=0.2, 
				   stop_words='english',
                                   use_idf=True, 
				   tokenizer=tokenize_and_stem, 
				   ngram_range=(1,3))

# note magic cmd %time
%time tfidf_matrix = tfidf_vectorizer.fit_transform(synopses_wiki)    # 6.05 secs

print(tfidf_matrix.shape)    # dimns of the tfidf matrix

C:\Users\31172\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Wall time: 6.81 s
(100, 218)


In [13]:
terms = tfidf_vectorizer.get_feature_names()
terms[:20]

['accept',
 'agre',
 'allow',
 'alon',
 'american',
 'ani',
 'anoth',
 'apart',
 'appear',
 'approach',
 'arm',
 'armi',
 'arrang',
 'arriv',
 'ask',
 'attack',
 'attempt',
 'away',
 'becaus',
 'becom']

In [17]:
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(),columns = terms)
#tfidf_df
tfidf_df

accept      agre     allow      alon  american       ani     anoth  \
0   0.196816  0.050128  0.000000  0.000000  0.000000  0.000000  0.000000   
1   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.183194   
2   0.000000  0.058951  0.142565  0.000000  0.079837  0.075890  0.000000   
3   0.000000  0.000000  0.000000  0.000000  0.101813  0.000000  0.000000   
4   0.000000  0.060724  0.073426  0.164476  0.082238  0.078172  0.000000   
5   0.000000  0.054107  0.065425  0.000000  0.109916  0.069654  0.000000   
6   0.051589  0.039418  0.000000  0.053384  0.080076  0.050745  0.091224   
7   0.000000  0.000000  0.090045  0.100852  0.151277  0.000000  0.000000   
8   0.000000  0.056360  0.000000  0.000000  0.000000  0.000000  0.000000   
9   0.000000  0.000000  0.000000  0.121230  0.000000  0.000000  0.000000   
10  0.000000  0.000000  0.000000  0.077812  0.077812  0.073964  0.000000   
11  0.033171  0.000000  0.061295  0.034326  0.000000  0.032629  0.000000   
12  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.104381   
13  0.086457  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
14  0.000000  0.064837  0.000000  0.000000  0.000000  0.000000  0.075025   
15  0.071187  0.054393  0.000000  0.000000  0.000000  0.000000  0.062939   
16  0.072979  0.055762  0.000000  0.000000  0.000000  0.000000  0.000000   
17  0.135305  0.155076  0.062505  0.000000  0.000000  0.000000  0.000000   
18  0.000000  0.045508  0.000000  0.061631  0.000000  0.000000  0.000000   
19  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
20  0.000000  0.000000  0.000000  0.076348  0.000000  0.000000  0.000000   
21  0.000000  0.065521  0.000000  0.000000  0.000000  0.000000  0.227448   
22  0.000000  0.068441  0.000000  0.000000  0.000000  0.000000  0.000000   
23  0.000000  0.000000  0.069484  0.000000  0.000000  0.000000  0.000000   
24  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
25  0.000000  0.070162  0.000000  0.000000  0.000000  0.000000  0.081186   
26  0.000000  0.034421  0.083243  0.046617  0.000000  0.000000  0.000000   
27  0.098495  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
28  0.000000  0.032830  0.000000  0.000000  0.000000  0.000000  0.037988   
29  0.000000  0.000000  0.107087  0.000000  0.209894  0.057005  0.000000   
..       ...       ...       ...       ...       ...       ...       ...   
70  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
71  0.160724  0.061403  0.000000  0.000000  0.000000  0.000000  0.071051   
72  0.000000  0.178788  0.000000  0.000000  0.000000  0.076720  0.000000   
73  0.180180  0.137673  0.000000  0.000000  0.000000  0.000000  0.000000   
74  0.058057  0.088720  0.000000  0.000000  0.000000  0.000000  0.000000   
75  0.000000  0.000000  0.079553  0.000000  0.000000  0.000000  0.152256   
76  0.069414  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
77  0.000000  0.041282  0.049917  0.055908  0.000000  0.000000  0.000000   
78  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.041564   
79  0.060893  0.000000  0.000000  0.000000  0.000000  0.239584  0.000000   
80  0.000000  0.035772  0.000000  0.000000  0.000000  0.000000  0.000000   
81  0.000000  0.000000  0.000000  0.000000  0.101701  0.000000  0.000000   
82  0.000000  0.078441  0.094849  0.000000  0.000000  0.000000  0.000000   
83  0.000000  0.000000  0.000000  0.083912  0.000000  0.039881  0.071695   
84  0.000000  0.000000  0.000000  0.000000  0.000000  0.076719  0.000000   
85  0.084361  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
86  0.049451  0.113355  0.045689  0.051172  0.000000  0.000000  0.131166   
87  0.000000  0.000000  0.068250  0.000000  0.000000  0.072662  0.000000   
88  0.000000  0.055874  0.000000  0.075670  0.000000  0.071928  0.000000   
89  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.090815   
90  0.000000  0.142422  0.000000  0.000000  0.000000  

Chalo, dassit for now. Back to the slides.

Sudhir